Factorization series: LU, QR, EVD, SVD
Solve Ax = b using the factorization

In [25]:
import numpy as np
from scipy.linalg import lu, solve_triangular
import time

In [28]:
# LU factorization

np.random.seed(0)
A = np.random.rand(2000,2000)
b = np.random.rand(2000)

start = time.time()
P,L,U = lu(A)
y = solve_triangular(L, P.T@b, lower = True)
x = solve_triangular(U,y)
end = time.time()
print(f" time: {end - start:.6f} sec")
print(x[0:5])
# print("검증:", np.allclose(L @U, P.T @ A))  # True여야 함


 time: 0.329490 sec
[ 0.0193996   0.27767683 -0.19615602 -0.16043055 -0.22259903]


In [33]:
## QR factorization

np.random.seed(0)
A = np.random.rand(2000,2000)
b = np.random.rand(2000)

start = time.time()
Q, R = np.linalg.qr(A)
# x = np.linalg.solve(R, Q.T @ b)
x = solve_triangular(R, Q.T@b)
end = time.time()
print(f" time: {end - start:.6f} sec")
print(x[0:5])

 time: 1.277308 sec
[ 0.0193996   0.27767683 -0.19615602 -0.16043055 -0.22259903]


In [32]:
## Eigenvalue decomposition

np.random.seed(0)
A = np.random.rand(2000,2000)
b = np.random.rand(2000)

start = time.time()
eigvals, eigvecs = np.linalg.eig(A)
x = eigvecs @ np.diag(1/eigvals) @ np.linalg.inv(eigvecs) @ b
end = time.time()
print(f" time: {end - start:.6f} sec")
print(x[0:5])

 time: 20.360008 sec
[ 0.0193996 -1.62830818e-13j  0.27767683-5.55947249e-13j
 -0.19615602+2.99175019e-13j -0.16043055-3.98084631e-15j
 -0.22259903+1.72152073e-13j]


In [34]:
## SVD

np.random.seed(0)
A = np.random.rand(2000,2000)
b = np.random.rand(2000)

start = time.time()
U, S, VT = np.linalg.svd(A)
x = VT.T @ np.diag(1/S) @ U.T @ b
end = time.time()
print(f" time: {end - start:.6f} sec")
print(x[0:5])

 time: 10.130722 sec
[ 0.0193996   0.27767683 -0.19615602 -0.16043055 -0.22259903]


In [40]:
## 직접 inverse matrix 구하는 것은 RREF로 해야함.
## library 의 inverse는 내부적으로 LU, 나 QR등의 optimized 된 형태임.

np.random.seed(0)
A = np.random.rand(2000,2000)
b = np.random.rand(2000)

start = time.time()
x = np.linalg.inv(A)@b
end = time.time()
print(f" time: {end - start:.6f} sec")
print(x[0:5])

 time: 0.852611 sec
[ 0.0193996   0.27767683 -0.19615602 -0.16043055 -0.22259903]


fast: LU > QR > SVD > EVD

stability: SVD > QR > LU > EVD



---
QR decomposition using a Modified Gram-schmidt Algorithm


In [42]:
def gram_schmidt(A):
  # A: (m x n) matrix

  n_rows, n_cols = A.shape
  Q = np.zeros((n_rows,n_cols))
  R = np.zeros((n_cols,n_cols))

  for i in range(n_cols):
    Qhat = A[:,i]

    for j in range(i):

      R[j, i] = np.dot(Q[:, j], A[:, i])
      Qhat = Qhat-R[j, i]*Q[:, j]

    R[i,i] = np.linalg.norm(Qhat)
    Q[:, i] = Qhat/ R[i,i]

  return Q, R

In [43]:
A = np.array([[1.0, 1.0, 1.0],
              [1.0, 1.0, 2.0],
              [1.0, 3.0, 3.0],
              [1.0, 2.0, 3.0]])

Q, R = gram_schmidt(A)

print("Q =\n", Q)
print("R =\n", R)
print("QᵗQ =\n", np.dot(Q.T, Q))  # Q^T Q ≈ I
print("QR =\n", np.dot(Q, R))     # QR ≈ A
print("A =\n", A)

Q =
 [[ 0.5        -0.45226702 -0.66742381]
 [ 0.5        -0.45226702  0.38138504]
 [ 0.5         0.75377836 -0.28603878]
 [ 0.5         0.15075567  0.57207755]]
R =
 [[2.         3.5        4.5       ]
 [0.         1.6583124  1.35680105]
 [0.         0.         0.95346259]]
QᵗQ =
 [[ 1.00000000e+00 -2.77555756e-17  0.00000000e+00]
 [-2.77555756e-17  1.00000000e+00  3.07033814e-16]
 [ 0.00000000e+00  3.07033814e-16  1.00000000e+00]]
QR =
 [[1. 1. 1.]
 [1. 1. 2.]
 [1. 3. 3.]
 [1. 2. 3.]]
A =
 [[1. 1. 1.]
 [1. 1. 2.]
 [1. 3. 3.]
 [1. 2. 3.]]



---
Eigenvalue decomposition


In [19]:
def compute_eigenvalues_2(A):
    a, b = A[0, 0], A[0, 1]
    c, d = A[1, 0], A[1, 1]

    # 특성 방정식의 계수
    x = a + d
    y = a * d - b * c

    # 근의 공식
    lambda1 = 1/2*(x) + np.sqrt(x**2 -4*y)/2
    lambda2 = 1/2*(x) - np.sqrt(x**2 -4*y)/2

    return lambda1, lambda2

def compute_eigenvector(A, lam, num):
  M = A - lam * np.eye(2)

  if num == 1:
    if abs(M[0,0]) > 1e-10:
      v = np.array([-M[0,1]/M[0,0],1])
    elif abs(M[1,0]) > 1e-10:
      v = np.array([-M[1,1]/M[1,0], 1])
    else:
      v = np.array([1, 0])
    v = [v]
  elif num == 2:
    if np.sum(A**2) < 1e-10:
      v1 = np.array([1,0])
      v2 = np.array([0,1])
      v = [v1, v2]
  return v


In [44]:
A = np.array([[1, 0], [0, 2]])

A = np.array([[2, 1],
              [1, 3]])

lambda1, lambda2 = compute_eigenvalues_2(A)
print(lambda1, lambda2)

if lambda1 == lambda2:
  v_list = compute_eigenvector(A, lambda1, 2)
  v1 = v_list[0]
  v2 = v_list[1]
else:
  v_list = compute_eigenvector(A, lambda1, 1)
  v1 = v_list[0]
  v_list = compute_eigenvector(A, lambda2, 1)
  v2 = v_list[0]

print(v1, v2)
print(v1.T@v2)


3.618033988749895 1.381966011250105
[0.61803399 1.        ] [-1.61803399  1.        ]
2.220446049250313e-16


Obtaining the largest eigenvalue and the corresponding eigenvector.

In [21]:
def power_iteration(A, max_iter=1000, tol=1e-8):
    n = A.shape[0]
    v = np.random.rand(n)  # 초기 벡터
    v = v / np.linalg.norm(v)  # 정규화

    for i in range(max_iter):
        Av = A @ v
        v_next = Av / np.linalg.norm(Av)

        # 수렴 검사
        if np.linalg.norm(v_next - v) < tol:
            break
        v = v_next

    # 고유값 근사 (Rayleigh quotient)
    eigenvalue = v.T @ A @ v
    return eigenvalue, v

In [22]:
A = np.array([[2, 4],
              [1, 3]])

eigval, eigvec = power_iteration(A)
print("Approx eigenvalue:", eigval)
print("Approx eigenvector:", eigvec)

# 검증: A @ v ≈ λ v
print("A @ v:", A @ eigvec)
print("lambda * v:", eigval * eigvec)

Approx eigenvalue: 4.561552839058643
Approx eigenvector: [0.84212294 0.5392856 ]
A @ v: [3.84138828 2.45997975]
lambda * v: [3.84138827 2.45997978]


QR Iteration for obtaining eigenvalues and eigenvectors.

In [46]:
def qr_iteration(A, max_iter=1000, tol=1e-10):
    A_k = A.copy()
    Q_total = np.eye(A.shape[0])
    for i in range(max_iter):
        Q, R = gram_schmidt(A_k)
        A_k = R @ Q
        Q_total = Q_total @ Q
        off_diag = A_k - np.diag(np.diag(A_k))
        if np.linalg.norm(off_diag) < tol:    # 수렴 검사 (대각 외 항이 거의 0이면)
            break
    return np.diag(A_k), Q_total

In [49]:
A = np.array([[2, 4],
              [1, 3]])

AA, V = qr_iteration(A)
print(AA, V)

print("A @ v:", A @V[:,0])
print("lambda * v:", AA[0] * V[:,0])

[4.56155281 0.43844719] [[ 0.84212294 -0.5392856 ]
 [ 0.5392856   0.84212294]]
A @ v: [3.84138826 2.45997973]
lambda * v: [3.84138826 2.45997973]
